In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from modules.faissvs import FAISSVectorStore
import pandas as pd
from prompts.system import FIRST_MATCHING_PARA_PLUS_FEWSHOTS
from modules.ollama_bot import fix_formatting_error_Ollama
from tqdm.notebook import tqdm

In [3]:
# Load the vector store (simulating a new session)
loaded_store = FAISSVectorStore.load_existing()

Initialized FAISSVectorStore with text-embedding-3-small, dimension=1536
✓ Loaded vector store from /home/user/alloy_comp_cleaning_rbcdsai/output/vector_store/
  - Total documents: 11


# Generating Responses

In [12]:
def get_llm_response(row):
    alloys = row['Alloy'].split(" | ")
    alloys = "\n".join(alloys)

    title = row['Title']
    abstract = row['Abstract']
    extracted_section = row['Paragraph']

    few_shots = loaded_store.get_context_for_llm(row, k=2, include_extra=True)
    
    if abstract and extracted_section != abstract:
        prompt = f"USER INPUT\n\nENTRIES_TO_MAP\n{alloys}\n\nTITLE\n{title}\n\nABSTRACT\n{abstract}\n\nEXTRACTED TEXT SECTION:\n{extracted_section}\n\n"
    else:
        prompt = f"USER INPUT\n\nENTRIES_TO_MAP\n{alloys}\n\nTITLE\n{title}\n\nEXTRACTED TEXT SECTION:\n{extracted_section}\n\n"

    prompt = few_shots + prompt

    prompt = prompt.strip()

    response = fix_formatting_error_Ollama(
        usr_prompt=prompt,
        system_prompt=FIRST_MATCHING_PARA_PLUS_FEWSHOTS
    )

    response = response.lstrip('```json')
    response = response.strip('```')
    response = response.replace('```', "")

    return response.strip()

In [5]:
df_full = pd.read_excel('./output/RegexMatchingGroups.xlsx')
df_full

,Doi,Alloy,Title,Paragraph,Abstract,Caption,Table_path,Footer
0,./10.1016.S0021-9614(03)00157-5,Cu-Ge | Al-Cu,Thermodynamics of liquid aluminium-copper-germ...,Partial for aluminium and integral enthalpies ...,Partial for aluminium and integral enthalpies ...,Evaluated parameters for the polynomial repres...,/home/shared/llm-extraction-pipeline/articles_...,{'a': 'The standard component state is liquid ...
1,./10.1016.S0021-9614(03)00157-5,Al-Ge,Thermodynamics of liquid aluminium-copper-germ...,The interpolation method[14]was used for prese...,Partial for aluminium and integral enthalpies ...,Evaluated parameters for the polynomial repres...,/home/shared/llm-extraction-pipeline/articles_...,{'a': 'The standard component state is liquid ...
2,./10.1016.S0021-9614(03)00157-5,Al-Cu-Ge,Thermodynamics of liquid aluminium-copper-germ...,Partial for aluminium and integral enthalpies ...,Partial for aluminium and integral enthalpies ...,The partial for aluminium and integral enthalp...,/home/shared/llm-extraction-pipeline/articles_...,NaN
3,./10.1016.S0021-9614(03)00157-5,Ge0.3Cu0.7-Al,Thermodynamics of liquid aluminium-copper-germ...,"Corresponding to the interpolation method, the...",Partial for aluminium and integral enthalpies ...,The evaluated a function coefficients of equat...,/home/shared/llm-extraction-pipeline/articles_...,NaN
4,./10.1016.S0040-6031(00)00502-5,Al17Ni17Si66-Cu,Thermodynamics of liquid and undercooled liqui...,The partial and the integral enthalpies of mix...,The partial and the integral enthalpies of mix...,Enthalpies of mixing of liquid Al17Ni17Si66-Cu...,/home/shared/llm-extraction-pipeline/articles_...,NaN
...,...,...,...,...,...,...,...,...
4591,./10.1016.j.tsf.2024.140220,Mo2,Effect of Mo content on the corrosion resistan...,Fig. 3a shows the planar SEM image of the Mo0 ...,(CoCrFeNi)1-xMox thin films with various Mo co...,Corrosion potential (Ecorr)corrosion current d...,/home/shared/llm-extraction-pipeline/articles_...,NaN
4592,./10.1016.j.tws.2020.107228,BMGs,Methods and materials for additive manufacturi...,BMGs or bulk amorphous alloys (BAAs) are solid...,Additive Manufacturing (AM) is the significant...,Properties of BMGs compared to metallic and gl...,/home/shared/llm-extraction-pipeline/articles_...,NaN
4593,./10.1016.j.tws.2020.107228,Ti-6Al-4V,Methods and materials for additive manufacturi...,Bush and Brice [193] have studied the characte...,Additive Manufacturing (AM) is the significant...,Summary of AM techniques in the fabrication of...,/home/shared/llm-extraction-pipeline/articles_...,NaN
4594,./10.1016.j.ultramic.2010.12.001,As-cast,Effect of decomposition of the Cr-Fe-Co rich p...,A new class of alloys with promising propertie...,"Splat-quenched, as-cast and aged (2h at 600°C ...","The saturation magnetization, remanance ratio ...",/home/shared/llm-extraction-pipeline/articles_...,NaN


In [9]:
df_test = df_full.iloc[20:31]

In [13]:
tqdm.pandas()
df_test['LLM Response w Rag'] = df_test.progress_apply(get_llm_response, axis=1)

  0%|          | 0/11 [00:00<?, ?it/s]

/tmp/ipykernel_437151/1608864414.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['LLM Response w Rag'] = df_test.progress_apply(get_llm_response, axis=1)


In [14]:
df_test.to_excel('./output/RegexMatchingGroup_Response-1.xlsx', index=False)